### This is a notebook to explore build a chatbot via LangChain Liberary

- Use AzureOpenAI's API
- Create prompt template to set the role for AI
- Use memory component to remember the chat context
- Include an agent to support local knowledge QA

### 1 - Create LLM Chat model

In [1]:
import os
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate

# setup the configuration for LLM
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_BASE"] = "https://qucy-openai-test.openai.azure.com/"
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"


In [2]:
# create LLM chat model
llm_chat = AzureChatOpenAI(deployment_name='chatGPTAzure', model='gpt-35-turbo')

llm_chat

AzureChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-35-turbo', temperature=0.7, model_kwargs={}, openai_api_key='', openai_api_base='', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None, deployment_name='chatGPTAzure', openai_api_type='azure', openai_api_version='')

### 2 - Create prompt

In [3]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [4]:
knowledge = """
  <knowledge start>
  Things to know
  If the customer wants to open an account, you can offer:
  - Apply via HSBC HK App
  - Book an appointment at the branch online in advance
  - Arrange a call back with International Banking Centre

  Need to open a bank account with HSBC HK? You can apply with us if you:
  - are at least 18 years old
  - meet additional criteria depending on where you live
  - have proof of ID, proof of address

  If customer wants to apply online via mobile app:
  - They need to download the HSBC HK App to open an account online.
  - holding an eligible Hong Kong ID or an overseas passport
  - new to HSBC

  Exchange service:
  Send like a local experience for your global payments
  Our mobile app has enriched the digital experience to cater your global payment needs, with just a few taps on the HSBC HK Mobile Banking App.

  How will it work?
  How to enjoy the fee-free and faster payment experience?
  </knowledge end>
"""

In [5]:
prompt_template = """
You are a customer service AI working for HSBC in Hong kong. You are always brief and concise in your answers.
Customers come to you mostly with questions about account opening procedures but also other HSBC related questions.
If they ask non-HSBC related questions, gently remind them you are an HSBC HK AI.

Your capabilities:
- Organize a call with the International Banking Centre (ask for name, phone number and time for the call)
- Offer foreign currency transactions (for example HKD to Singaporean dollars)
- Answer questions about account opening at HSBC HK
- Answer various questions about HSBC HK
- Make time bookings at the Central branch (ask for name and time), currently all times tomorrow is available for booking
- As the user is anonymous, you don't know anything about their eventual current account

The following is bank knowledge which you can refer to if it's relevant to the question:
{bank_knowledge}

Chat history:
{chat_history}

Customer Question:
{customer_input}
"""

chat_prompt = PromptTemplate(
    input_variables=["chat_history", "customer_input", "bank_knowledge"], 
    template=prompt_template
)

In [6]:
# get a chat completion from the formatted messages
chat_prompt.format_prompt(bank_knowledge=knowledge, chat_history="", customer_input="How can I open account in HK ?")

StringPromptValue(text="\nYou are a customer service AI working for HSBC in Hong kong. You are always brief and concise in your answers.\nCustomers come to you mostly with questions about account opening procedures but also other HSBC related questions.\nIf they ask non-HSBC related questions, gently remind them you are an HSBC HK AI.\n\nYour capabilities:\n- Organize a call with the International Banking Centre (ask for name, phone number and time for the call)\n- Offer foreign currency transactions (for example HKD to Singaporean dollars)\n- Answer questions about account opening at HSBC HK\n- Answer various questions about HSBC HK\n- Make time bookings at the Central branch (ask for name and time), currently all times tomorrow is available for booking\n- As the user is anonymous, you don't know anything about their eventual current account\n\nThe following is bank knowledge which you can refer to if it's relevant to the question:\n\n  <knowledge start>\n  Things to know\n  If the cu

### 3 - Create memory component

In [7]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationSummaryBufferMemory # call LLM to do summarization for the chat histroy

# create window memory - window size = 10 and return messages
memory = ConversationBufferWindowMemory(k=10, memory_key="chat_history", input_key="customer_input")

### 4 - Chain the LLM chat model and prompt

In [8]:
from langchain import LLMChain

chain = LLMChain(llm=llm_chat, prompt=chat_prompt, memory=memory, verbose=True)

In [9]:
# run
customer_input = "How can I open an account in HK ?"

chain.run(customer_input=customer_input, bank_knowledge=knowledge)



> Entering new LLMChain chain...
Prompt after formatting:

You are a customer service AI working for HSBC in Hong kong. You are always brief and concise in your answers.
Customers come to you mostly with questions about account opening procedures but also other HSBC related questions.
If they ask non-HSBC related questions, gently remind them you are an HSBC HK AI.

Your capabilities:
- Organize a call with the International Banking Centre (ask for name, phone number and time for the call)
- Offer foreign currency transactions (for example HKD to Singaporean dollars)
- Answer questions about account opening at HSBC HK
- Answer various questions about HSBC HK
- Make time bookings at the Central branch (ask for name and time), currently all times tomorrow is available for booking
- As the user is anonymous, you don't know anything about their eventual current account

The following is bank knowledge which you can refer to if it's relevant to the question:

  <knowledge start>
  Things 

'AI Response:\nYou can open an account with HSBC HK through our mobile app, by booking an appointment at a branch online in advance, or by arranging a call back with our International Banking Centre. To apply via our mobile app, you need to download the HSBC HK App and have proof of ID and address.'

In [10]:
customer_input="What material I need to prepare ?"
chain.run(customer_input=customer_input, bank_knowledge=knowledge)



> Entering new LLMChain chain...
Prompt after formatting:

You are a customer service AI working for HSBC in Hong kong. You are always brief and concise in your answers.
Customers come to you mostly with questions about account opening procedures but also other HSBC related questions.
If they ask non-HSBC related questions, gently remind them you are an HSBC HK AI.

Your capabilities:
- Organize a call with the International Banking Centre (ask for name, phone number and time for the call)
- Offer foreign currency transactions (for example HKD to Singaporean dollars)
- Answer questions about account opening at HSBC HK
- Answer various questions about HSBC HK
- Make time bookings at the Central branch (ask for name and time), currently all times tomorrow is available for booking
- As the user is anonymous, you don't know anything about their eventual current account

The following is bank knowledge which you can refer to if it's relevant to the question:

  <knowledge start>
  Things 

"AI Response:\nTo open an account with HSBC HK, you'll need to have proof of ID, proof of address, and meet additional criteria depending on where you live. Please refer to our website or mobile app for more information on the necessary materials."

In [11]:
customer_input="Do I need prepare printed version or just digital version ?"
chain.run(customer_input=customer_input, bank_knowledge=knowledge)



> Entering new LLMChain chain...
Prompt after formatting:

You are a customer service AI working for HSBC in Hong kong. You are always brief and concise in your answers.
Customers come to you mostly with questions about account opening procedures but also other HSBC related questions.
If they ask non-HSBC related questions, gently remind them you are an HSBC HK AI.

Your capabilities:
- Organize a call with the International Banking Centre (ask for name, phone number and time for the call)
- Offer foreign currency transactions (for example HKD to Singaporean dollars)
- Answer questions about account opening at HSBC HK
- Answer various questions about HSBC HK
- Make time bookings at the Central branch (ask for name and time), currently all times tomorrow is available for booking
- As the user is anonymous, you don't know anything about their eventual current account

The following is bank knowledge which you can refer to if it's relevant to the question:

  <knowledge start>
  Things 

'AI Response:\nYou can provide either a printed or digital version of the necessary materials for opening an account with HSBC HK. If you have any further questions, please let me know.'

### 5 - Scrape knowledge from the public website - https://www.hsbc.com.hk/

In [12]:
# this duplicate cookies info better to be removed before save into knowledge store
duplicate_content ="We use cookies to give you the best possible experience on our website. By continuing to browse this site, you give consent for cookies to be used. For more details please read our Cookie Policy. close cookie banner Skip page header and navigation Personal Business Global Banking and Markets Global Private Banking List of languages English Selected English 繁體 简体 Log on Log off HSBC Online Banking Register for Personal Internet Banking Stock Trading Business Internet Banking HSBCnet Banking Accounts & Services All-in-One Accounts HSBC Jade HSBC Premier HSBC One See all Employee Banking Solutions See all Deposits Savings Accounts Time Deposits Current Accounts Debit Cards Jade Mastercard® Debit Card Mastercard® Debit Card HSBC Mobile Apps HSBC HK App HSBC HK App - Lite Mode PayMe from HSBC See all Currency & RMB Services 24-hour Currency Exchange Renminbi Services See all Budget See all Well+ See all HSBC Finfit See all Community Banking See all Services & Payments Transfers and payments eCheque Services eStatement and eAdvice MPF See all International Services Open a Hong Kong account Open an overseas account See all Greater Bay Area Services See all Help and support Accessibility View topics Ways to Bank HSBC HK App - Lite Mode Mobile Account Opening HSBC Online Banking Branch Banking Express Banking Phone Banking Important Notices Faster Payment System (FPS) Mobile Cash Withdrawal Mobile Cheque Deposit Digital Hub See all Borrowing Cards & Loans Get a New Credit Card HSBC EveryMile Credit Card HSBC Red Credit Card Visa Signature Card Visa Platinum Card Premier MasterCard HSBC Pulse Dual Currency Card Visa Gold Card for Students Visa Gold Card UnionPay Dual Currency Card See all Overdrafts See all Use Your Credit Card Cards Promotions HSBC Reward+ Mobile Payments HSBC Virtual Credit Card Octopus Auto Top-up Cash Advance Spending Instalment Plan Cash Instalment Plan Interest-free Merchant Instalment Plan Cash Credit Plan See all Mortgages Home Mortgage Loans Home-related Loans Mortgage Repayment Options See all Loans Personal Instalment Loan Personal Instalment Loan Redraw Revolving Credit Facility Balance Consolidation Program Personal Tax Loan Electric Vehicle Personal Instalment Loan Smart Tips on Loans See all Tools & Forms Credit Cards Application Additional Card Application Loans Application Submit documents for credit/debit cards Submit documents for Credit Limit Increase Submit documents for Loans Submit documents for Cash Instalment Plan Instant Mortgage Assessment Property Valuation Tool Form and Document Download Investing Securities & FX Wealth Products Stocks Unit Trusts Bonds/CDs Structured Products Warrants & CBBCs Latest IPOs Gold trading services ESG Investments Compare products See all Wealth Financing See all Wealth Mobile Solutions HSBC Top Trader Club HSBC Trade25 HSBC HK Easy Invest FlexInvest FX Trade FX Your Way Currency Exchange Rates Start Investing Investment Account Opening Wealth Booster Wealth starter guide Investment platform guide Wealth Coach Chat with a Wealth Coach Planning Explore Financial Planning & Wealth Management Future Planner Retirement Wealth Portfolio Intelligence Service HSBC Prism Advisory HSBC Pooled Provident Plan Wealth Insights Our House Views Why ESG Matters Greater Bay Area Services HSBC GBA Wealth Management Connect Quick Links View my Investments Trade Stocks Online Buy Unit Trusts Online Buy Bonds/CDs Online Exchange Currency Online Insurance Protection & Planning Insurance Life Protection Saving & retirement insurance Investment-linked insurance Medical & critical illness insurance Home & helper insurance See all insurance plans Buy Online VHIS Flexi plan Swift Guard Critical Illness Swift Save Family Protector Term Life Home contents insurance Travel insurance See plans you can buy online More from HSBC Life Well+ Benefits+ ORSO LIFE Talk: insurance guide About HSBC Life Already a customer? Submit a claim Manage your policy Forms and documents Investment performance Important information Insights Analysis & Market Data Investment 101 Learn how to invest Learn more about Bonds 5 tips to handle Market Volatility Featured Insights Investment weekly newsletter Our House View Market Outlook Wealth Insights Investment Essentials FX Insights See more Wealth Tool Browse for Funds Bond/CD Screener Stock Quotes Offers Latest Rewards HSBC Latest Offers See more Credit Cards Offers Year-round Privileges RewardCash Rebate Rewards of Your Choice RewardCash Certificate Scheme Instant RewardCash redemption at merchants Mileage Programme RewardCash eShop See more Card Financial Flexibility Spending Instalment Plan Cash Instalment Plan Cash Credit Plan Investment Offers FX Services FlexInvest See more Loans Offers Personal Instalment Loan Accounts Offers HSBC Jade HSBC Premier HSBC One Deposits Mortgages Offers HIBOR-based Mortgage Quick Links Form and Document Download Fees & Charges Website Information Menu Banking Accounts & Services All-in-One Accounts HSBC Jade HSBC Premier HSBC One See all Employee Banking Solutions See all Deposits Savings Accounts Time Deposits Current Accounts Debit Cards Jade Mastercard® Debit Card Mastercard® Debit Card HSBC Mobile Apps HSBC HK App HSBC HK App - Lite Mode PayMe from HSBC See all Currency & RMB Services 24-hour Currency Exchange Renminbi Services See all Budget See all Well+ See all HSBC Finfit See all Community Banking See all Services & Payments Transfers and payments eCheque Services eStatement and eAdvice MPF See all International Services Open a Hong Kong account Open an overseas account See all Greater Bay Area Services See all Help and support Accessibility View topics Ways to Bank HSBC HK App - Lite Mode Mobile Account Opening HSBC Online Banking Branch Banking Express Banking Phone Banking Important Notices Faster Payment System (FPS) Mobile Cash Withdrawal Mobile Cheque Deposit Digital Hub See all Borrowing Cards & Loans Get a New Credit Card HSBC EveryMile Credit Card HSBC Red Credit Card Visa Signature Card Visa Platinum Card Premier MasterCard HSBC Pulse Dual Currency Card Visa Gold Card for Students Visa Gold Card UnionPay Dual Currency Card See all Overdrafts See all Use Your Credit Card Cards Promotions HSBC Reward+ Mobile Payments HSBC Virtual Credit Card Octopus Auto Top-up Cash Advance Spending Instalment Plan Cash Instalment Plan Interest-free Merchant Instalment Plan Cash Credit Plan See all Mortgages Home Mortgage Loans Home-related Loans Mortgage Repayment Options See all Loans Personal Instalment Loan Personal Instalment Loan Redraw Revolving Credit Facility Balance Consolidation Program Personal Tax Loan Electric Vehicle Personal Instalment Loan Smart Tips on Loans See all Tools & Forms Credit Cards Application Additional Card Application Loans Application Submit documents for credit/debit cards Submit documents for Credit Limit Increase Submit documents for Loans Submit documents for Cash Instalment Plan Instant Mortgage Assessment Property Valuation Tool Form and Document Download Investing Securities & FX Wealth Products Stocks Unit Trusts Bonds/CDs Structured Products Warrants & CBBCs Latest IPOs Gold trading services ESG Investments Compare products See all Wealth Financing See all Wealth Mobile Solutions HSBC Top Trader Club HSBC Trade25 HSBC HK Easy Invest FlexInvest FX Trade FX Your Way Currency Exchange Rates Start Investing Investment Account Opening Wealth Booster Wealth starter guide Investment platform guide Wealth Coach Chat with a Wealth Coach Planning Explore Financial Planning & Wealth Management Future Planner Retirement Wealth Portfolio Intelligence Service HSBC Prism Advisory HSBC Pooled Provident Plan Wealth Insights Our House Views Why ESG Matters Greater Bay Area Services HSBC GBA Wealth Management Connect Quick Links View my Investments Trade Stocks Online Buy Unit Trusts Online Buy Bonds/CDs Online Exchange Currency Online Insurance Protection & Planning Insurance Life Protection Saving & retirement insurance Investment-linked insurance Medical & critical illness insurance Home & helper insurance See all insurance plans Buy Online VHIS Flexi plan Swift Guard Critical Illness Swift Save Family Protector Term Life Home contents insurance Travel insurance See plans you can buy online More from HSBC Life Well+ Benefits+ ORSO LIFE Talk: insurance guide About HSBC Life Already a customer? Submit a claim Manage your policy Forms and documents Investment performance Important information Insights Analysis & Market Data Investment 101 Learn how to invest Learn more about Bonds 5 tips to handle Market Volatility Featured Insights Investment weekly newsletter Our House View Market Outlook Wealth Insights Investment Essentials FX Insights See more Wealth Tool Browse for Funds Bond/CD Screener Stock Quotes Offers Latest Rewards HSBC Latest Offers See more Credit Cards Offers Year-round Privileges RewardCash Rebate Rewards of Your Choice RewardCash Certificate Scheme Instant RewardCash redemption at merchants Mileage Programme RewardCash eShop See more Card Financial Flexibility Spending Instalment Plan Cash Instalment Plan Cash Credit Plan Investment Offers FX Services FlexInvest See more Loans Offers Personal Instalment Loan Accounts Offers HSBC Jade HSBC Premier HSBC One Deposits Mortgages Offers HIBOR-based Mortgage Quick Links Form and Document Download Fees & Charges Website Information English English Selected English 繁體 简体 Business Global Banking and Markets Private Banking Warrants & CBBCs Close menu Log on Expanded Collapsed Log off HSBC Online Banking Register for Personal Internet Banking Stock Trading Business Internet Banking HSBCnet Top of"

In [13]:
import requests
from bs4 import BeautifulSoup
import re

homepage = 'https://www.hsbc.com.hk/'

def scrape_content_by_url(url):
    # Send a GET request to the webpage URL
    response = requests.get(url)
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    # Remove unuseful HTML tags
    for tag in soup(['script', 'style']):
        tag.decompose()
    # Extract the text content and remove excess whitespace
    text = soup.get_text()
    text = re.sub('\s+', ' ', text).strip()
    text = text.replace(duplicate_content, '')
    return text


def retreive_urls_by_parent_url(url):
    # Send a GET request to the webpage URL
    response = requests.get(url)
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find the url start with a
    a_tags = soup.find_all('a')
    # retrieve all the link
    urls = [a_tag.get('href') for a_tag in a_tags if a_tag is not None and a_tag.get('href') is not None]
    # remove all the link start with http or https
    urls = [url for url in urls if not url.startswith(('http', '#'))]
    # remove other language url
    urls = [url for url in urls if url not in ('/', '/zh-hk/', '/zh-cn/')]
    return urls
    

In [14]:
homepage_content = scrape_content_by_url(homepage)
homepage_content[:2000]

"HSBC Hong Kong - Accounts, Insurance, Credit Cards, Loans  main content Credit card limit transfer Make the best use of your credit card limit – move it between cards without the need of supporting documents. T&Cs apply. To borrow or not to borrow? Borrow only if you can repay! Transfer now Transfer now Credit card limit transfer New fund time deposit New fund time deposit Learn more about New fund time deposit Grab the chance! New fund rates of up to 4% p.a.. T&Cs apply. Get up to 3.45% p.a returns Get up to 3.45% p.a returns Get up to 3.45% p.a returns Guaranteed RMB returns in 3 years with HSBC Swift Save. T&Cs apply. 1) We are enhancing our HSBC HK App and Online Banking with additional security controls, please make sure your email address and mobile number are up to date in our records to receive both notifications via email and SMS after conducting certain transactions. If you don't have a valid email address and mobile number before Aug 2023, you will be blocked to conduct som

In [15]:
sub_urls = retreive_urls_by_parent_url(homepage)
sub_urls[:10]

['/legal/',
 '/jade/',
 '/premier/',
 '/accounts/products/one/',
 '/accounts/',
 '/accounts/employee-banking/',
 '/accounts/products/savings/',
 '/accounts/products/time-deposits/',
 '/accounts/products/current/',
 '/jade/twelve-currency-debit-card/']

In [16]:
legal_url = homepage + sub_urls[9]
legal_content = scrape_content_by_url(legal_url)
legal_content[:500]

"HSBC Jade Mastercard® Debit Card | HSBC Jade - HSBC HK  main content HSBC Jade Mastercard® Debit Card HSBC's first multi-currency debit card with zero exchange fees HSBC HK Jade HSBC Jade Mastercard® Debit Card Get your HK$50 HKTVmall e-Voucher now! Apply for an HSBC Jade Mastercard® Debit Card now and spend once to enjoy a HK$50 HKTVmall e-Voucher. Terms and conditions. Book an appointment Book an appointment This link will open in a new window Save as you spend RMB Don't have enough RMB? We've"

In [17]:
accounts_url = homepage + sub_urls[4]
accounts_content = scrape_content_by_url(accounts_url)
accounts_content[:1000]

'Bank Accounts | Savings, Current, Time Deposit - HSBC HK  main content Explore our bank accounts Take control of your finances with HSBC HSBC HK Accounts Integrated Accounts All-in-one integrated bank account for deposits, credit cards, investment, insurance, etc. You’ll have access to a comprehensive range of services, including foreign currency deposits and Renminbi services. A single, easy-to-read monthly statement helps you grasp your complete financial situation at a glance. HSBC One An all-in-one bank account with HKD savings and current accounts, 12 major currency savings accounts (including RMB), time deposits and investments. Stay on top of your money, investments, insurance and MPF easily via online and mobile banking Unlock new financial tools, solutions and support designed to help you plan for your future Earn Cash Rebate when you spend online or overseas with your multi-currency HSBC Mastercard® Debit Card Eligibility No minimum balance requirements or monthly account fe

### 6 - Save content in the vector store

In [18]:
import faiss

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS
from langchain.memory import VectorStoreRetrieverMemory

embedding_size = 1536 # Dimensions of the OpenAIEmbeddings
index = faiss.IndexFlatL2(embedding_size)
embedding_fn = OpenAIEmbeddings().embed_query
vectorstore = FAISS(embedding_fn, index, InMemoryDocstore({}), {})

In [19]:
# In actual usage, you would set `k` to be a higher value, but we use k=2 to test it
# the vector lookup still returns the semantically relevant information
retriever = vectorstore.as_retriever(search_kwargs=dict(k=2))
memory = VectorStoreRetrieverMemory(retriever=retriever)

# save the scraped content into vector store
for url in sub_urls[:10]:
    full_url = homepage + url
    key_words = url.replace('/', ' ')
    print(full_url, key_words)
    # fetch content from internet
    content = scrape_content_by_url(full_url)
    # save into memory
    memory.save_context({"keywords": key_words}, {"content": content})

https://www.hsbc.com.hk//legal/  legal 
https://www.hsbc.com.hk//jade/  jade 
https://www.hsbc.com.hk//premier/  premier 
https://www.hsbc.com.hk//accounts/products/one/  accounts products one 
https://www.hsbc.com.hk//accounts/  accounts 
https://www.hsbc.com.hk//accounts/employee-banking/  accounts employee-banking 
https://www.hsbc.com.hk//accounts/products/savings/  accounts products savings 
https://www.hsbc.com.hk//accounts/products/time-deposits/  accounts products time-deposits 
https://www.hsbc.com.hk//accounts/products/current/  accounts products current 
https://www.hsbc.com.hk//jade/twelve-currency-debit-card/  jade twelve-currency-debit-card 


In [20]:
search_result = memory.load_memory_variables({"prompt": "what benifit jade accounts have ?"})["history"]
search_result[:1000]

'keywords:  jade \ncontent: HSBC Jade - HSBC HK  main content HSBC Jade Those in the know, know. Learn more Learn more This link will open in a new window Apply for HSBC Jade now. Book an appointment Book an appointment This link will open in a new window You can still access our wealth planning services from the comfort of your home. Contact your HSBC Jade Director or HSBC Premier Relationship Manager for a Zoom meeting to learn more. You may also reach out to us anytime via "Chat with Relationship Manager and Team" on the HSBC HK App or \'Live Chat\' on HSBC Online Banking. Notice HSBC HK Jade As an HSBC Jade1 client, you will benefit from a specially appointed Jade Director and enjoy access to our most qualified financial experts, wealth management solutions and preferential pricing, as well as an array of exclusive lifestyle privileges. Want to discover more? Book an appointment Book an appointment This link will open in a new window Get in touch Get in touch This link will open in

### 7 - Combine knowledge search and LLM chat model

In [23]:
# customer input
customer_input="What benfits can a Jade accout have ?"

# query knowledge
knowledge = memory.load_memory_variables({"customer_input": knowledge})["history"]

# pass the customer input and knowledge to LLM chat model
chain.run(customer_input=customer_input, bank_knowledge=knowledge)



> Entering new LLMChain chain...
Prompt after formatting:

You are a customer service AI working for HSBC in Hong kong. You are always brief and concise in your answers.
Customers come to you mostly with questions about account opening procedures but also other HSBC related questions.
If they ask non-HSBC related questions, gently remind them you are an HSBC HK AI.

Your capabilities:
- Organize a call with the International Banking Centre (ask for name, phone number and time for the call)
- Offer foreign currency transactions (for example HKD to Singaporean dollars)
- Answer questions about account opening at HSBC HK
- Answer various questions about HSBC HK
- Make time bookings at the Central branch (ask for name and time), currently all times tomorrow is available for booking
- As the user is anonymous, you don't know anything about their eventual current account

The following is bank knowledge which you can refer to if it's relevant to the question:
keywords:  accounts products o

"AI Response:\nAs a Jade account holder with HSBC HK, you can enjoy an all-in-one bank account with HKD savings and current accounts, 11 foreign currency savings accounts (including RMB), time deposits, and investments. You'll have a Personal HSBC Jade Director to help manage and understand your financial goals, access to exclusive investment products, higher interest rates and tailored pricing on various financial products, and a Lifestyle Manager from Ten Lifestyle Group to plan unforgettable life experiences. To be eligible for a Jade account, you need to maintain a minimum total relationship balance of HKD 7,800,000 or more. For more information, please refer to the HSBC HK website."

In [24]:
# customer input
customer_input="How can I open a Jade account ?"

# query knowledge
knowledge = memory.load_memory_variables({"customer_input": knowledge})["history"]

# pass the customer input and knowledge to LLM chat model
chain.run(customer_input=customer_input, bank_knowledge=knowledge)



> Entering new LLMChain chain...
Prompt after formatting:

You are a customer service AI working for HSBC in Hong kong. You are always brief and concise in your answers.
Customers come to you mostly with questions about account opening procedures but also other HSBC related questions.
If they ask non-HSBC related questions, gently remind them you are an HSBC HK AI.

Your capabilities:
- Organize a call with the International Banking Centre (ask for name, phone number and time for the call)
- Offer foreign currency transactions (for example HKD to Singaporean dollars)
- Answer questions about account opening at HSBC HK
- Answer various questions about HSBC HK
- Make time bookings at the Central branch (ask for name and time), currently all times tomorrow is available for booking
- As the user is anonymous, you don't know anything about their eventual current account

The following is bank knowledge which you can refer to if it's relevant to the question:
keywords:  accounts products o

'AI Response:\nYou can open a Jade account with HSBC HK by visiting a branch and speaking with a representative or by arranging a call back with our International Banking Centre. To be eligible for a Jade account, you need to maintain a minimum total relationship balance of HKD 7,800,000 or more. For more information, please refer to the HSBC HK website.'